In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
from  sklearn.metrics import mean_squared_error

In [2]:

target_df = pd.DataFrame()
temp = [23, 22+1/3,22+2/3,22, 22,22,22, 22+2/3,22+4/3,24, 24+2/3,24+4/3,26,27,28,27, 
        26+2/3,26+1/3,26,25,24,23,23,23]

#https://www.weather.go.kr/weather/forecast/timeseries.jsp

wind_degree = np.repeat([90,90,90,90,90,90,90,90], 3)
wind_speed = np.repeat([4,4,4,7,7,7,7,4], 3)
precipitation = np.repeat([0], 24)
humidity = np.repeat([95,95,100,90,80,75, 80, 95], 3)

In [3]:
target_df["temperature"] = temp
target_df["precipitation"] = precipitation
target_df["wind_degree"] = wind_degree
target_df["wind_speed"] = wind_speed
target_df["humidity"] = humidity

In [21]:
scaler = MinMaxScaler().fit(target_df)


In [22]:
dfs = []
for filename in glob.glob("*.csv"):
    dfs.append(pd.read_csv(filename, encoding="euc-kr")[['일시','기온(°C)', '강수량(mm)', '풍향(16방위)','풍속(m/s)', '습도(%)']])

KeyError: "['강수량(mm)', '풍향(16방위)'] not in index"

In [ ]:
for i in range(4):
    dfs[i] =dfs[i].fillna(0)
    dfs[i] = dfs[i].rename(columns={'일시':'datetime','기온(°C)':"temperature", '강수량(mm)':"precipitation",
                           '풍향(16방위)':"wind_degree",'풍속(m/s)':"wind_speed", '습도(%)':"humidity"})
    dfs[i] = dfs[i].set_index("datetime")



In [ ]:
min_error = 9999
min_year = 0
min_date = 0
idx =0
for y in [19, 18, 16, 17]:
    for date in range(1, 9):
        temp = scaler.transform(dfs[idx][f"20{y}-08-0{date} 00:00":f"20{y}-08-0{date} 23:00"])
        if min_error > (mean_squared_error(temp, target_df.values)):
            min_error = mean_squared_error(temp, target_df.values)
            min_year = y
            min_date = date
    idx+=1
print(min_error, min_year, min_date)


In [ ]:
min_error = 9999
min_year = 0
min_date = 0
idx =0
for y in [19, 18, 16, 17]:
    for date in range(24,32):
        temp = scaler.transform(dfs[idx][f"20{y}-07-{date} 00:00":f"20{y}-07-{date} 23:00"])
        if min_error > (mean_squared_error(temp, target_df.values)):
            min_error = mean_squared_error(temp, target_df.values)
            min_year = y
            min_date = date
    idx+=1
print(min_error, min_year, min_date)
        

In [29]:
#2017 8 3
y = 17
date =3 
idx = 3
base = pd.read_csv("SURFACE_ASOS_131_MI_2017-08_2017-08_2017.csv", encoding='euc-kr').fillna(0)
base.columns

base = base[['일시', '기온(°C)','풍속(m/s)', '습도(%)','일사(MJ/m^2)','일조(Sec)']]
base = base.rename(columns={'일시':'datetime', '기온(°C)':"temperature",'풍속(m/s)':"wind_speed", '습도(%)':"humidity", 
                            "일조(Sec)":"solar_radiation",'일사(MJ/m^2)':"solar_intensity"  
})
base.datetime = pd.to_datetime(base.datetime, infer_datetime_format=True)
base["difference"] = base["datetime"].sub(pd.to_datetime("2017-01-01", infer_datetime_format=True), axis=0)/ np.timedelta64(1, 'D')

base = base.set_index('datetime')
base = base[f"20{y}-08-0{date} 00:00":f"20{y}-08-0{date} 23:59"]




In [31]:
out = pd.DataFrame()
period_min = 15
out["result"] = 0
out["temperature"] = base["temperature"].resample(f'{str(period_min)}T').mean()
out["wind_speed"] = base["wind_speed"] .resample(f'{str(period_min)}T').mean()
out["humidity"]= base["humidity"].resample(f'{str(period_min)}T').mean()
out["solar_intensity"] = base["solar_intensity"].resample(f'{str(period_min)}T').mean().diff()
out["solar_radiation"] = base["solar_radiation"].resample(f'{str(period_min)}T').mean().diff()
out["date"] = 31
out["month"] = 7
out["hour"] = pd.DatetimeIndex(out.index).hour
out["difference"] = base["difference"]
out.loc[out["solar_intensity"] < 0 , "solar_intensity"] = 0
out.loc[out["solar_radiation"] < 0 , "solar_radiation"] = 0
out = out.fillna(0)